Modifying the LCS code shared in the notebook below will significantly speed up the process.  
https://www.kaggle.com/code/ryotayoshinobu/foursquare-lightgbm-baseline

Evaluation using `name` (578907 entries) in `pairs_df.csv` showed a speedup from 25.7 seconds to 3.2 seconds.

Will be helpful if you evaluate a lot of candidate points!

In [ ]:
import pandas as pd

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [ ]:
%%cython
import numpy as np  # noqa
cpdef int FastLCS(str S, str T):
    cdef int i, j
    cdef int cost
    cdef int v1,v2,v3,v4
    cdef int[:, :] dp = np.zeros((len(S) + 1, len(T) + 1), dtype=np.int32)
    for i in range(len(S)):
        for j in range(len(T)):
            cost = (int)(S[i] == T[j])
            v1 = dp[i, j] + cost
            v2 = dp[i + 1, j]
            v3 = dp[i, j + 1]
            v4 = dp[i + 1, j + 1]
            dp[i + 1, j + 1] = max((v1,v2,v3,v4))
    return dp[len(S)][len(T)]

In [ ]:
pairs_df = pd.read_csv('../input/foursquare-location-matching/pairs.csv')
pairs_df = pairs_df.fillna("")
print(len(pairs_df),"pairs")

# Since dataframes are expensive to retrieve, they are stored in tuple in advance.
names_1 = tuple(pairs_df["name_1"])
names_2 = tuple(pairs_df["name_2"])

In [ ]:
%%timeit
# Original LCS 
# 25.7 s ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
for name_1,name_2 in zip(names_1,names_2):
    d1 = LCS(name_1,name_2)

In [ ]:
%%timeit
# FastLCS
# 3.21 s ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
for name_1,name_2 in zip(names_1,names_2):
    d2 = FastLCS(name_1,name_2)

In [ ]:
# Check that all values match
for name_1,name_2 in zip(names_1,names_2):
    d1 = FastLCS(name_1,name_2)
    d2 = LCS(name_1,name_2)
    if d1!=d2:
        print("ng")
        break
else:
    print("ok")